In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'train', 'train.csv', 'sample_submission.csv']


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,Flatten,Dropout,MaxPool2D
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
import cv2
import glob
from tqdm import tqdm

Using TensorFlow backend.


In [3]:
df = pd.read_csv('../input/train.csv')

In [4]:
df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [6]:
im = cv2.imread('../input/train/train/'+df['id'][0])

In [7]:
im.shape

(32, 32, 3)

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.15,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [9]:
df['has_cactus'] = df['has_cactus'].astype(str)

In [10]:
train_generator = train_datagen.flow_from_dataframe(df,directory='../input/train/train/',subset='training',x_col='id',y_col = 'has_cactus',target_size = (32,32),class_mode='binary')
test_generator = train_datagen.flow_from_dataframe(df,directory='../input/train/train/',subset='validation',x_col='id',y_col = 'has_cactus',target_size = (32,32),class_mode='binary')

Found 14875 images belonging to 2 classes.
Found 2625 images belonging to 2 classes.


In [11]:
model = Sequential()

In [12]:
model.add(Convolution2D(32,(3,3),activation='relu',input_shape = (32,32,3)))
model.add(Convolution2D(32,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Convolution2D(64,(3,3),activation='relu'))
model.add(Convolution2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Convolution2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
model.add(Flatten())
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation = 'sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics=['accuracy'])

In [15]:
history = model.fit_generator(train_generator,steps_per_epoch=2000,epochs=10,validation_data=test_generator,validation_steps=64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2000/2000 [==============================] - 131s 65ms/step - loss: 0.1267 - acc: 0.9513 - val_loss: 0.0833 - val_acc: 0.9702
Epoch 2/10
2000/2000 [==============================] - 106s 53ms/step - loss: 0.0593 - acc: 0.9781 - val_loss: 0.0464 - val_acc: 0.9826
Epoch 3/10
2000/2000 [==============================] - 109s 55ms/step - loss: 0.0469 - acc: 0.9832 - val_loss: 0.0630 - val_acc: 0.9772
Epoch 4/10
2000/2000 [==============================] - 105s 52ms/step - loss: 0.0411 - acc: 0.9847 - val_loss: 0.0371 - val_acc: 0.9861
Epoch 5/10
2000/2000 [==============================] - 105s 53ms/step - loss: 0.0348 - acc: 0.9876 - val_loss: 0.0279 - val_acc: 0.9902
Epoch 6/10
2000/2000 [==============================] - 109s 54ms/step - loss: 0.0296 - acc: 0.9890 - val_loss: 0.0849 - val_acc: 0.9732
Epoch 7/10
2000/2000 [==============================] - 105s 53ms/step - loss: 0.0295 - acc: 0.9897 - val_loss: 0.0376 - val_acc: 

In [16]:
test = glob.glob('../input/test/test/*.jpg')

In [17]:
submission = pd.read_csv('../input/sample_submission.csv')
submission.head()

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,0.5
1,0017242f54ececa4512b4d7937d1e21e.jpg,0.5
2,001ee6d8564003107853118ab87df407.jpg,0.5
3,002e175c3c1e060769475f52182583d0.jpg,0.5
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.5


In [18]:
test_path = '../input/test/test/'
test_images_names = []

for filename in os.listdir(test_path):
    test_images_names.append(filename)
    
test_images_names.sort()

images_test = []

for image_id in tqdm(test_images_names):
    images_test.append(np.array(cv2.imread(test_path + image_id)))
    
images_test = np.asarray(images_test)
images_test = images_test.astype('float32')
images_test /= 255

100%|██████████| 4000/4000 [00:06<00:00, 577.94it/s]


In [19]:
prediction = model.predict(images_test)

In [20]:
predict = []
for i in tqdm(range(len(prediction))):
    if prediction[i][0]>0.5:
        answer = prediction[i][0]
    else:
        answer = prediction[i][0]
    predict.append(answer)

100%|██████████| 4000/4000 [00:00<00:00, 325992.73it/s]


In [21]:
submission['has_cactus'] = predict

In [22]:
submission.head(50)

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,1.000000
1,0017242f54ececa4512b4d7937d1e21e.jpg,1.000000
2,001ee6d8564003107853118ab87df407.jpg,0.000513
3,002e175c3c1e060769475f52182583d0.jpg,0.051883
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.999987
5,0038ed6f7417b8275b393307f66cb1e3.jpg,1.000000
6,0059dfa41de4441fc23b9a4b98ba99cb.jpg,1.000000
7,0061bd302d735db829418cb90b4e6040.jpg,1.000000
8,00754573d8c7223e73da553dd43780bd.jpg,1.000000
9,008d9fce71c8e03a4c9cdd15ea908573.jpg,0.001093


In [24]:
submission.to_csv('sample_submission.csv',index = False)